In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import functools
import math
import collections

import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sobol_seq
import scipy.stats

from bvbq_torch import distributions,utils
from bvbq_torch import BVBQMixMVN

ImportError: cannot import name 'BVBQNamedMixMVN' from 'bvbq_torch.named_interface' (../bvbq_torch/named_interface.py)

In [ ]:
scipy.stats.gamma(1.0).rvs(size=(10,1))

In [ ]:
def loggaussian(x,mu,sigma):
    return -0.5*((x-mu)/sigma)**2 - np.log(np.sqrt(2*np.pi)*sigma)

def baselogprob(params):
    mu,sigma = params['mu'],params['sigma']
    term1 = np.log(scipy.stats.gamma(3.0).pdf(sigma))
    term2 = loggaussian(mu,0,sigma)
    return term1 + term2

def basesample(n):
    sigma = scipy.stats.gamma(1.0).rvs(size=(n,1))
    mu = np.random.randn(n,1)*sigma
    res = {'mu':mu,'sigma':sigma}
    return res


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
samples = basesample(1000)
ax.scatter(samples['mu'],samples['sigma'])
ax.set_xlabel('mu')
ax.set_ylabel('sigma')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

sigma_plot,mu_plot = np.meshgrid(np.linspace(0,7,101)[1:],np.linspace(-8,8))
params = {'mu':mu_plot,'sigma':sigma_plot}
zplot = baselogprob(params)
ax.plot_surface(mu_plot,sigma_plot,np.exp(zplot))

In [ ]:
bvbq = BVBQMixMVN(['mu','sigma'],[1,1],[(None,None),(0.0,None)])

In [ ]:
ndata = 50
mu = np.random.randn(ndata,1)
sigma = scipy.stats.gamma(5).rvs(size=(ndata,1))
params_init = {'mu':mu,'sigma':sigma}
params_evals = baselogprob(params_init)
bvbq.initialize_data(params_init,params_evals,empirical_params=True)
bvbq.initialize_components(mean=torch.zeros(2),var=20*torch.ones(2))

In [ ]:
for i in range(30):
    bvbq.update_distribution()

In [ ]:
distrib = bvbq.distribution

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
samples = basesample(1000)
samples_vb = distrib.sample(1000)
ax.scatter(samples['mu'],samples['sigma'],color='blue',alpha=0.3)
ax.scatter(samples_vb['mu'].numpy(),samples_vb['sigma'].numpy(),color='red',alpha=0.3)
ax.set_xlabel('mu')
ax.set_ylabel('sigma')

In [ ]:
fig = plt.figure(figsize=(16,16))
ax1 = fig.add_subplot(221,projection='3d')
ax2 = fig.add_subplot(222,projection='3d')
ax3 = fig.add_subplot(223,projection='3d')
sigma_plot,mu_plot = np.meshgrid(np.linspace(0,7,101)[1:],np.linspace(-8,8))
zplot3 = baselogprob({'mu':mu_plot,'sigma':sigma_plot})
zplot2 = distrib.logprob({'mu':mu_plot[...,None],'sigma':sigma_plot[...,None]}).numpy()
zplot3 = bvbq.surrogate_prediction({'mu':mu_plot[...,None],'sigma':sigma_plot[...,None]}).numpy()
ax1.plot_surface(mu_plot,sigma_plot,np.exp(zplot),color='blue',alpha=1.0)
ax2.plot_surface(mu_plot,sigma_plot,np.exp(zplot2),color='red',alpha=1.0)
ax3.plot_surface(mu_plot,sigma_plot,np.exp(zplot3),color='green',alpha=1.0)

In [ ]:
bvbq.surrogate_prediction({'mu':mu_plot[...,None],'sigma':sigma_plot[...,None]})